In [1]:
#Run the code with single GPU
##launch your code with command line of '''python MLM.py'''

# Run the code with multi GPU(fast)
## 1.set your accelerate setting with command line of '''accelerate config'''
## 2.launch your code with command line of '''accelerate launch MLM.py'''
## Offical tutorial: https://github.com/huggingface/accelerate


In [2]:
import torch
torch.cuda.empty_cache()

c:\Users\lin\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
!nvidia-smi

Wed Jun  7 12:36:32 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 517.48       Driver Version: 517.48       CUDA Version: 11.7     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ... WDDM  | 00000000:01:00.0  On |                  N/A |
|  0%   34C    P8    21W / 200W |   1586MiB /  8192MiB |     10%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [4]:
import torch
import math
import os
from transformers import AutoTokenizer, AutoModelForMaskedLM,AutoConfig
from accelerate import Accelerator
import logging
from tqdm import trange, tqdm
from transformers import (
    AdamW,
    get_scheduler
)

In [5]:
import json
from pprint import pprint

In [6]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [8]:
output_dir ='MLM'

In [9]:
input_fold="Stock"
import os
import json
files=os.listdir(input_fold)
textjson=[]
for i in files:
    with open(input_fold+"\\"+i, "r",encoding='utf-8') as f:
        a_data=json.load(f)
        for j in a_data:
            textjson.append(j['content'])

In [10]:
#output_dir = input("output_dir = ")

# Load model and tokenizer
# model_name = "bert-base-chinese"
model_name = "hfl/chinese-roberta-wwm-ext"
#model_dir=
tokenizer = AutoTokenizer.from_pretrained(model_name, model_max_length=512)
config = AutoConfig.from_pretrained("config.json")
model = AutoModelForMaskedLM.from_pretrained("pytorch_model.bin",config=config).to(device)


# Load corpus

#with open("C:\\Users\\lin\\Desktop\\gitcode\\NTCIR\\train_social2.json", "r", encoding="utf-8") as f:
#    textjson = json.load(f)
#print("sample:\n", textjson[0], "\n\n")

In [11]:
print(textjson[0])

請問哪裡可以查詢三大法人歷史期貨留倉紀錄阿？

我在期交所最多只能查到近三年，不知道哪裡可以看到更久遠的資料，如果可以希望可以
看到十年以上

包括大台、小台、金電等等

在此先謝過各路股海好漢了

--


In [12]:
#print(textjson[0][0])

In [13]:
# converting into str
text=[]
inputs = tokenizer(textjson, return_tensors="pt", padding=True, truncation=True, stride = 256)
inputs["labels"] = inputs.input_ids.detach().clone()
## detach() method in PyTorch is used to separate a tensor from the computational graph by returning a new tensor that doesn't require a gradient. 
## clone() function makes an exact copy of the original image. One can use this clone image to manipulate without affecting the original image. 
if "labels" in inputs.keys():
    print("Process Label success!\n\n")

Process Label success!




In [14]:
# Process inputs(MASK) of training data
rand = torch.rand(inputs.input_ids.shape)
print("rand.shape: ", rand.shape)
print("sample of rand: ", rand, "\n\n")

mask_probabiltiy = 0.15
mask_arr = (rand < mask_probabiltiy) * (inputs.input_ids != 101) * \
            (inputs.input_ids != 102) * (inputs.input_ids != 0)
##　101->[CLS]	102->[SEP]  0->[PAD]
print("Bool mask_arr:", mask_arr)

selection = []
for i in range(mask_arr.shape[0]):
    selection.append(
        torch.flatten(mask_arr[i].nonzero()).tolist()
    )
for i in range(mask_arr.shape[0]):
    inputs.input_ids[i, selection[i]] = 103  # 103->[MASK]
print("inputs.input_ids", inputs.input_ids, "\n\n")

rand.shape:  torch.Size([76005, 512])
sample of rand:  tensor([[0.5807, 0.2963, 0.9611,  ..., 0.4211, 0.4758, 0.0786],
        [0.6870, 0.5875, 0.3137,  ..., 0.4834, 0.7009, 0.7893],
        [0.0748, 0.3728, 0.1534,  ..., 0.4799, 0.1735, 0.0333],
        ...,
        [0.8442, 0.2164, 0.3182,  ..., 0.0435, 0.3245, 0.9109],
        [0.0993, 0.6097, 0.5465,  ..., 0.5884, 0.5694, 0.5081],
        [0.1519, 0.6073, 0.2454,  ..., 0.9247, 0.2706, 0.2327]]) 


Bool mask_arr: tensor([[False, False, False,  ..., False, False, False],
        [False, False, False,  ..., False, False, False],
        [False, False, False,  ..., False, False, False],
        ...,
        [False, False, False,  ..., False, False, False],
        [False, False, False,  ..., False, False, False],
        [False, False, False,  ..., False, False, False]])
inputs.input_ids tensor([[ 101, 6313, 1558,  ...,    0,    0,    0],
        [ 101,  673, 3299,  ...,    0,    0,    0],
        [ 101, 2207, 2475,  ...,    0,    0,  

In [15]:
# Form of Dataset and Dataloader 
class MeditationsDataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings
    def __getitem__(self, idx):
        return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
    def __len__(self):
        return len(self.encodings.input_ids)

dataset = MeditationsDataset(inputs)
dataloader = torch.utils.data.DataLoader(dataset, batch_size = 2, shuffle = True)


In [22]:
# Parameter of training
train_batch_size = 1024
num_train_epochs = 32     
learning_rate=3e-5        
gradient_accumulation_steps = 1

no_decay = ["bias", "LayerNorm.weight"]
optimizer_grouped_parameters = [
    {
        "params": [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)],
        "weight_decay": 0.0,
    },                                
    {
        "params": [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)],
        "weight_decay": 0.0,
    },
]
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)

## Scheduler and math around the number of training steps.
num_update_steps_per_epoch = math.ceil(len(dataloader) / gradient_accumulation_steps)
max_train_steps = num_train_epochs * num_update_steps_per_epoch
print('max_train_steps', max_train_steps)

## scheduler
lr_scheduler = get_scheduler(
    name="linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=max_train_steps,
)

## Initialize the accelerator. We will let the accelerator handle device
accelerator = Accelerator()

## Prepare everything with our `accelerator`.
model, optimizer, dataloader = accelerator.prepare(
    model, optimizer, dataloader
)

max_train_steps 1216096


In [25]:
# Train
logger = logging.getLogger(__name__)
logging.basicConfig(
    format="%(asctime)s - %(levelname)s - %(name)s - %(message)s",
    datefmt="%m/%d/%Y %H:%M:%S",
    level=logging.INFO,
)
logger.info(accelerator.state)

total_batch_size = train_batch_size * accelerator.num_processes * gradient_accumulation_steps

logger.info("***** Running training *****")
logger.info(f"  Num examples = {len(dataset)}")
logger.info(f"  Num Epochs = {num_train_epochs}")
logger.info(f"  Instantaneous batch size per device = {train_batch_size}")
logger.info(f"  Total train batch size (w. parallel, distributed & accumulation) = {total_batch_size}")
logger.info(f"  Gradient Accumulation steps = {gradient_accumulation_steps}")
logger.info(f"  Total optimization steps = {max_train_steps}")

def train(model, dataloader, max_train_steps, lr_scheduler, num_train_epochs, gradient_accumulation_steps, optimizer):
    completed_steps = 0
    best_epoch = {}
    
    #if output_dir not in os.listdir():
    #    os.mkdir(output_dir)

    for epoch in trange(num_train_epochs, desc="Epoch"):
        model.train()
        for step, batch in enumerate(tqdm(dataloader, desc="Iteration")):
            outputs = model(batch["input_ids"], attention_mask = batch["attention_mask"], labels = batch["labels"])
            loss = outputs.loss
            loss = loss / gradient_accumulation_steps
            accelerator.backward(loss)
            if step % gradient_accumulation_steps == 0 or step == len(dataloader) - 1:
                optimizer.step()
                lr_scheduler.step()
                optimizer.zero_grad()
                completed_steps += 1

            if step % 1000 == 0:
                print({'epoch': epoch, 'step': step, 'loss': loss.item()})

            if completed_steps >= max_train_steps:
                break

        
        accelerator.wait_for_everyone()
        unwrapped_model = accelerator.unwrap_model(model)
        unwrapped_model.save_pretrained(output_dir + '/epoch_' + str(epoch+19) + '/', save_function=accelerator.save)
    

06/07/2023 12:49:40 - INFO - __main__ - Distributed environment: NO
Num processes: 1
Process index: 0
Local process index: 0
Device: cuda
Mixed precision type: no

06/07/2023 12:49:40 - INFO - __main__ - ***** Running training *****
06/07/2023 12:49:40 - INFO - __main__ -   Num examples = 76005
06/07/2023 12:49:40 - INFO - __main__ -   Num Epochs = 32
06/07/2023 12:49:40 - INFO - __main__ -   Instantaneous batch size per device = 1024
06/07/2023 12:49:40 - INFO - __main__ -   Total train batch size (w. parallel, distributed & accumulation) = 1024
06/07/2023 12:49:40 - INFO - __main__ -   Gradient Accumulation steps = 1
06/07/2023 12:49:40 - INFO - __main__ -   Total optimization steps = 1216096


In [26]:
if __name__ == '__main__':
    train(model, dataloader, max_train_steps, lr_scheduler, num_train_epochs, 
          gradient_accumulation_steps, optimizer)

Epoch:   0%|          | 0/32 [00:00<?, ?it/s]C:\Users\lin\AppData\Local\Temp\ipykernel_20336\1417647038.py:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


{'epoch': 0, 'step': 0, 'loss': 0.1907489150762558}


{'epoch': 0, 'step': 1000, 'loss': 0.11359620094299316}


{'epoch': 0, 'step': 2000, 'loss': 0.24448923766613007}


{'epoch': 0, 'step': 3000, 'loss': 0.08173388987779617}


...

{'epoch': 19, 'step': 28000, 'loss': 0.02929244190454483}


Iteration:  74%|███████▎  | 28003/38003 [1:07:44<21:42,  7.68it/s]

{'epoch': 19, 'step': 29000, 'loss': 0.014169168658554554}


{'epoch': 19, 'step': 30000, 'loss': 0.009752277284860611}


{'epoch': 19, 'step': 31000, 'loss': 0.01773066818714142}
